# Soil Audit:
### outputs:
* Check Soil APN status vs structual status and current location in smartsheets
* No Soil Sample Dates in smartsheets


## Downloads needed:
* 2 Smartsheets both Central and South or BOP and ANVIL
* Soil Tracker - Sharepoint

# Libraries

In [1]:
import pandas as pd
import os
import glob
import numpy as np
from datetime import timedelta
import smartsheet

# Varables

In [2]:
csv = '.csv'
excel = '.xlsx'

In [3]:
while True:
    user = input('What project are we Auditing?: Central or Northern ').lower()
    try:

        #central Set up
        if user == 'central':
            print('Central Audit Started')
            # Counties:
            county= ['ALPINE', 'EL DORADO', 'LAKE', 'NEVADA', 'PLACER', 'TUOLUMNE', 'MENDOCINO', 'TULARE', 'KERN']

            # Division. * northern goes here becasue we want everything not in northern Division
            division = 'Northern Division'

            # smartsheets Shet IDs
            sheet_id1 = '1844440261257092' # central
            sheet_id2= '6292262490531716' # south central

            # Central's API
            API1 = smartsheet.Smartsheet('6jJFxnAfFsDErU0BSr1Ae4D6cHq0zenczVR12') # API central

            API1.errors_as_exceptions(True)

            # Central's API
            API2 = smartsheet.Smartsheet("H8mqAo60J2zGHSrJEwmlMoY88A9yjgCo6RnSI") # API south central
            API2.errors_as_exceptions(True)
            break

        #northern Set up
        elif user == 'northern':
            print('Northern Audit Started')
            # Counties:
            county = ['LASSEN', 'PLUMAS', 'SISKIYOU', 'TRINITY', 'SHASTA', 'TEHAMA']

            # Division. * northern goes here becasue we want everything not in northern Division
            division = 'Central Division'

            # smartsheets Shet IDs
            sheet_id1 = '3589680131270532' # BOP
            sheet_id2= '5367491607193476' # ANVIL

            # Central's API
            API1 = smartsheet.Smartsheet('6w7vusWNdvdm9TJmjImZfbh0SPQgi3HJaekNb') # API BOP

            API1.errors_as_exceptions(True)

            # Central's API
            API2 = smartsheet.Smartsheet("MQQmCjvvFWLKBoX9K7A33rHSoLnuj1MGVJuVK") # API ANVIL
            API2.errors_as_exceptions(True)
            break
    except ValueError:
        print("\ninvaild input Only 'Central or Northern' allowed")

What project are we Auditing?: Central or Northern northern
Northern Audit Started


# Function to set up Smartsheet into a readable Database

In [4]:
def smartsheet_loadup(sheet):
    columns = [col.title for col in sheet.columns]
    rows = []
    for row in sheet.rows:
        cells = []
        for cell in row.cells:
            cells.append(cell.value)
        rows.append(cells)
    df = pd.DataFrame(rows, columns=columns)
    return df

Area1_SS = smartsheet_loadup(API1.Sheets.get_sheet(sheet_id1))
Area2_SS = smartsheet_loadup(API2.Sheets.get_sheet(sheet_id2))

df_both = pd.concat([Area1_SS,Area2_SS])

# Read in Soil Tracker and Clean up

In [5]:
path_soilTracker = r"C:\Users\jacque.trahan\Downloads\Soil Sampling Tracker*"
load_soil = glob.glob(path_soilTracker + csv)
soils_sharePoint = pd.read_csv(max(load_soil, key=os.path.getctime))

soils_sharePoint = soils_sharePoint[~(soils_sharePoint['Division'].isin([division]))]

In [6]:
# Sharepoint Columns
soils_sharePoint = soils_sharePoint[['APN', 'APN Status', 'Field Notes', 'Tt Recommendation','Date Collected',
                                                                              "Date Lab Recd",
                                                                       'State Recommendation', 'State Rec Details',
                                                                       'DTSC Mercury Site','State Recommendation',
                                                                      'State Rec Details','State Rec Date','QC State Recs','ExpectedResultsDate',
                                                                      'Current Sample Event','Division', 'Recs Notes']]

In [7]:
# Smartsheet column set up
SS_soils = df_both.copy()
SS_soils = SS_soils[['APN_ROW Segment','Street #', 'Street Name', 'County','Structural Status','Debris Crew',
                                   'Soil Sample', 'Soil Sample Results Rec\'d from Lab',"Soil Sample Results",
                                  'Soil Boring','Soil Boring Results Rec\'d from Lab', 'Soil Boring Results',
                                  'Soil Re-scrape',"Re-scrape Results Rec'd from Lab", "Re-scrape Results",
                                  'Soil Re-scrape 2',"Re-scrape 2 Results Rec'd from Lab", "Re-scrape 2 Results",
                                  'Soil Rescrape 3',"Re-scrape 3 Results Rec'd from Lab", 'Re-scrape 3 Results',
                                  "Soil Samples Approved"]]


# Fuction to check location of last date entered in smartsheets

In [8]:
def soil_location(df):
    """Prints out column header if the cell is not blank"""
    if pd.notna(df["Soil Samples Approved"]):
        return "Soil Samples Approved"
    
    elif pd.notnull(df['Re-scrape 3 Results']):
        return 'Re-scrape 3 Results'
    
    elif pd.notnull(df["Re-scrape 3 Results Rec'd from Lab"]):
        return "Re-scrape 3 Results Rec'd from Lab"
    
    elif pd.notnull(df["Re-scrape 2 Results"]):
        return "Re-scrape 2 Results"
    
    elif pd.notnull(df["Re-scrape 2 Results Rec'd from Lab"]):
        return "Re-scrape 2 Results Rec'd from Lab"
    
    elif pd.notnull(df['Soil Re-scrape 2']):
        return 'Soil Re-scrape 2'
    
    elif pd.notnull(df["Re-scrape Results"]):
        return "Re-scrape Results"
    
    elif pd.notnull(df["Re-scrape Results Rec'd from Lab"]):
        return "Re-scrape Results Rec'd from Lab"
    
    elif pd.notnull(df['Soil Re-scrape']):
        return 'Soil Re-scrape'
    
    elif pd.notnull(df['Soil Boring Results']):
        return 'Soil Boring Results'
    
    elif pd.notnull(df['Soil Boring Results Rec\'d from Lab']):
        return 'Soil Boring Results Rec\'d from Lab'
    
    elif pd.notnull(df['Soil Boring']):
        return 'Soil Boring'
    
    elif pd.notnull(df["Soil Sample Results"]):
        return "Soil Sample Results"
    
    elif pd.notnull(df['Soil Sample Results Rec\'d from Lab']):
        return 'Soil Sample Results Rec\'d from Lab'
    
    elif pd.notnull(df['Soil Sample']):
        return 'Soil Sample'
    

In [9]:
SS_soils.insert(SS_soils.columns.get_loc("Soil Samples Approved")+1, "Location in smartsheets",
                      SS_soils.apply(soil_location, axis=1))



# Merge Smasrtsheets and Soil Tracker adn Clean up

In [17]:
Soil_final = SS_soils.merge(soils_sharePoint,
                              how='left',
                              left_on='APN_ROW Segment',
                              right_on='APN')

# change names of columns

In [18]:
Soil_final.rename(columns={'Soil Sample Results Rec\'d from Lab': '1st Lab sample received-lab',
                          'Soil Boring Results Rec\'d from Lab': 'Boring 1st result received-lab',
                          "Re-scrape Results Rec'd from Lab": 'rescrape 1st recived-lab',
                          "Re-scrape 2 Results Rec'd from Lab": '2nd rescrape-lab',
                          "Re-scrape 3 Results Rec'd from Lab": "3rd rescrape-lab"}, inplace=True)

# Function to check if dates are greater than another date adn should not be

In [19]:
def Soil_Sample(df):
    if df['Soil Sample'] > df['1st Lab sample received-lab']:
        return 'Check Soil Sample Dates'
    else:
        return 'Match'
    
def Soil_Boring(df):
    if df['Soil Boring'] > df['Boring 1st result received-lab']:
        return 'Check Soil Boring Dates'
    else:
        return 'Match'
    
def Soil_Rescrape(df):
    if df['Soil Re-scrape'] > df['rescrape 1st recived-lab']:
        return 'Check Soil Re-scrape Dates'
    else:
        return 'Match'
def Soil_Rescrape2(df):    
    if df['Soil Re-scrape 2'] > df['2nd rescrape-lab']:
        return 'Check Soil Re-scrape 2 Dates'
    else:
        return 'Match'

def Soil_Rescrape3(df):    
    if df['Soil Rescrape 3'] > df["3rd rescrape-lab"]:
        return 'Check Soil Rescrape 3 Dates'
    else:
        return 'Match'

In [20]:
# Final Column set up. Column order
Soil_final = Soil_final.copy()
Soil_final = Soil_final[['APN_ROW Segment','Street #', 'Street Name', 'County','Structural Status','Debris Crew',
                         "Location in smartsheets", 'APN Status',
                         'Soil Sample','Date Collected', '1st Lab sample received-lab',"Soil Sample Results",
                         'Recs Notes','Current Sample Event','ExpectedResultsDate',"Soil Samples Approved",
                         
                      'Soil Boring','Boring 1st result received-lab', 'Soil Boring Results',
                         
                      'Soil Re-scrape',"rescrape 1st recived-lab", "Re-scrape Results",
                                             
                      'Soil Re-scrape 2',"2nd rescrape-lab", "Re-scrape 2 Results",
                                                
                      'Soil Rescrape 3',"3rd rescrape-lab", 'Re-scrape 3 Results',
                         
                        'Field Notes', 'Tt Recommendation', 'State Recommendation', 'State Rec Details',
                         'DTSC Mercury Site',
                         'State Rec Details','State Rec Date','QC State Recs',
                         'Division']]

# Final Data clean up after merge

In [21]:
Soil_final['Soil Sample'] = pd.to_datetime(Soil_final['Soil Sample']).fillna(pd.NaT)
Soil_final['1st Lab sample received-lab'] = pd.to_datetime(Soil_final['1st Lab sample received-lab']).fillna(pd.NaT)

Soil_final['Soil Re-scrape 2'] = pd.to_datetime(Soil_final['Soil Re-scrape 2']).fillna(pd.NaT)
Soil_final["2nd rescrape-lab"] = pd.to_datetime(Soil_final["2nd rescrape-lab"]).fillna(pd.NaT)

Soil_final['Soil Rescrape 3'] = pd.to_datetime(Soil_final['Soil Rescrape 3']).fillna(pd.NaT)
Soil_final["3rd rescrape-lab"] = pd.to_datetime(Soil_final["3rd rescrape-lab"]).fillna(pd.NaT)

Soil_final['Soil Boring'] = pd.to_datetime(Soil_final['Soil Boring']).fillna(pd.NaT)
Soil_final['Boring 1st result received-lab'] = pd.to_datetime(Soil_final['Boring 1st result received-lab']).fillna(pd.NaT)

Soil_final['Soil Re-scrape'] = pd.to_datetime(Soil_final['Soil Re-scrape']).fillna(pd.NaT)
Soil_final["rescrape 1st recived-lab"] = pd.to_datetime(Soil_final["rescrape 1st recived-lab"]).fillna(pd.NaT)


# Audit Check for matches

In [22]:
# sample date check
Soil_final.insert(Soil_final.columns.get_loc("Soil Samples Approved")+1, 'Soil_Sample Date checks-M',
                 Soil_final.apply(Soil_Sample, axis=1))

# Soil_Boring Check
Soil_final.insert(Soil_final.columns.get_loc('Soil Boring Results')+1, 'Soil_Boring Date checks-M',
                 Soil_final.apply(Soil_Boring, axis=1))

#Soil_Rescrape
Soil_final.insert(Soil_final.columns.get_loc("Re-scrape Results")+1, 'Soil_Rescrape Date checks-M',
                 Soil_final.apply(Soil_Rescrape, axis=1))

#Soil_Rescrape2
Soil_final.insert(Soil_final.columns.get_loc("Re-scrape 2 Results")+1, 'Soil_Rescrape2 Date checks-M',
                 Soil_final.apply(Soil_Rescrape2, axis=1))

#Soil_Rescrape3
Soil_final.insert(Soil_final.columns.get_loc('Re-scrape 3 Results')+1, 'Soil_Rescrape3 Date checks-M',
                 Soil_final.apply(Soil_Rescrape3, axis=1))

Soil_final.set_index('APN_ROW Segment', inplace=True)
Soil_final.to_excel('Check Soil APN status vs structual status and current location in smartsheets.xlsx')

# No sample date in SS, But sampled in Soil tracker

In [23]:
Soil_final = Soil_final[(Soil_final['Soil Sample'].isnull()) & (Soil_final['Date Collected'].notnull())]

Soil_final = Soil_final[['Street #', 'Street Name', 'County', 'Structural Status', 'Debris Crew',
        'APN Status', 'Soil Sample','Date Collected']]

# Soil_Sample_date_checks.set_index('APN_ROW Segment', inplace=True)
Soil_final.to_excel('No Soil Sample Dates in smartsheets.xlsx')